In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np

# 载入数据集
mnist = input_data.read_data_sets("MNIST_data",one_hot=True)

# 定义每个批次的大小
batch_size = 100
# 计算一共有多少个批次
n_batch = mnist.train.num_examples // batch_size

# # 根据样本样式定义创建占位符（1列）
x = tf.placeholder(tf.float32,[None,784]) # 每张图的像素点信息： 28*28 = 784
y = tf.placeholder(tf.float32,[None,10])  # 十个数字
keep_prob = tf.placeholder(tf.float32) # keep_prod 每个元素被保留的概率，那么 keep_prob:1就是所有元素全部保留的意思
lr =  tf.Variable(0.001, dtype = tf.float32) # 学习率变量

# 定义神经网络的中间层
# 
# （输入层784个数据，中间层500个神经元）
W1 = tf.Variable(tf.truncated_normal([784,500],stddev=0.1))
b1 =  tf.Variable(tf.zeros([500])+0.1)
# 激活函数：tanh
L1 = tf.nn.tanh(tf.matmul(x,W1) + b1)
L1_drop =  tf.nn.dropout(L1,keep_prob) # 防止过拟合


W2 = tf.Variable(tf.truncated_normal([500,300],stddev=0.1)) 
b2 =  tf.Variable(tf.zeros([300])+0.1)
L2 = tf.nn.tanh(tf.matmul(L1_drop,W2) + b2)
L2_drop =  tf.nn.dropout(L2,keep_prob)


W4 = tf.Variable(tf.truncated_normal([300,10],stddev=0.1)) 
b4 =  tf.Variable(tf.zeros([10])+0.1)
L4 = tf.matmul(L2_drop,W4) + b4
prediction = tf.nn.softmax(L4)


# 二次代阶函数 回归问题
# loss = tf.reduce_mean(tf.square(y - prediction))

# 交叉熵 分类问题
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels = y, logits = prediction))

# 定义一个梯度下降法来进行训练的优化器
train =  tf.train.AdamOptimizer(lr).minimize(loss)

# 结果存放在一个布尔型的列表中
correct_prediction = tf.equal(tf.arg_max(y,1),tf.arg_max(prediction,1))# arg_max 返回一维张量中最大值的所以在位置
# 求准确率
# cast 把布尔型列表转换为float32， 如[true.true.false] =》 [1,1,0] ，那么准确率的值即为66.6%
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

with tf.Session() as sess:
    # 初始化全部变量
    sess.run(tf.global_variables_initializer())
    # 训练21个周期
    for epoch in range(51):
        # 每个周期都改变学习率（学习越久学习率越低）
        sess.run(tf.assign(lr, 0.001*(0.95**epoch)))
        # n_batch ： 一共有多少个批次
        for batch in range(n_batch):
            # 保存batch_size张图片的数据与标签
            batch_xs,batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train,feed_dict={x:batch_xs,y:batch_ys,keep_prob:1.0})

        # 用测试集的图片及标签求得准确率
        test_acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_prob:1.0})
        train_acc = sess.run(accuracy,feed_dict={x:mnist.train.images,y:mnist.train.labels,keep_prob:1.0})
        print("Iter "+ str(epoch) + ", Texting acc " + str(test_acc)+ ", train_acc acc " + str(train_acc))


keep_prob:1.0

意思是每个元素被保留的概率，那么 keep_prob:1就是所有元素全部保留的意思。
一般在大量数据训练时，为了防止过拟合，添加Dropout层，设置一个0~1之间的小数